In [ ]:
   "import numpy as np\n",
    "import pandas as pd\n",
    "import seaborn as sns\n",
    "import matplotlib.pyplot as plt\n",
    "\n",
    "import bokeh.io\n",
    "import bokeh.plotting\n",
    "import bokeh.models\n",
    "\n",
    "import holoviews as hv\n",
    "hv.extension(\"bokeh\")\n",
    "\n",
    "import panel as pn\n",
    "pn.extension()\n",
    "\n",
    "bokeh.plotting.output_notebook()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "id": "ad61d52c-00f2-4483-89ac-a832cf294a5f",
   "metadata": {},
   "outputs": [],
   "source": [
    "df = pd.read_excel(\"Halfman_Systems SerologyP1_100121 PH.xlsx\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "id": "23a8ea33-440e-4fc5-8038-f803c3304743",
   "metadata": {},
   "outputs": [],
   "source": [
    "ag_lst = [name.split(\"_\")[0] for name in df.columns[2:]]\n",
    "ab_fcr_lst = [name.split(\"_\")[1] for name in df.columns[2:]]\n",
    "\n",
    "ag_select = pn.widgets.Select(name='Select Antigen', options=list(np.unique(ag_lst)))\n",
    "ab_fcr_select = pn.widgets.Select(name='Select Ig or FcR', options=list(np.unique(ab_fcr_lst)))\n",
    "\n",
    "ig_fcr_choose = pn.widgets.RadioButtonGroup(options=['Ig Titer', 'FcR Binding'], button_type='success')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "id": "e42dc3e5-4f2f-438d-862e-34c7dbb61fbd",
   "metadata": {},
   "outputs": [],
   "source": [
    "df_plot = df.melt(id_vars=[\"Treatment\", \"Sample\"])\n",
    "\n",
    "ags, igs_fcrs = list(zip(*df_plot.variable.str.split(\"_\")))\n",
    "\n",
    "df_plot[\"Ag\"] = ags\n",
    "df_plot[\"Ig_FcR\"] = igs_fcrs"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "id": "fb85a7c2-31a9-4ea0-b77c-4a2c1b9014ef",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Treatment</th>\n",
       "      <th>Sample</th>\n",
       "      <th>variable</th>\n",
       "      <th>value</th>\n",
       "      <th>Ag</th>\n",
       "      <th>Ig_FcR</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>Control VLP</td>\n",
       "      <td>H45</td>\n",
       "      <td>B.1.1.7 S-2p_IgA</td>\n",
       "      <td>2599.75</td>\n",
       "      <td>B.1.1.7 S-2p</td>\n",
       "      <td>IgA</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>Control VLP</td>\n",
       "      <td>H45L</td>\n",
       "      <td>B.1.1.7 S-2p_IgA</td>\n",
       "      <td>2980.00</td>\n",
       "      <td>B.1.1.7 S-2p</td>\n",
       "      <td>IgA</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>Control VLP</td>\n",
       "      <td>H46</td>\n",
       "      <td>B.1.1.7 S-2p_IgA</td>\n",
       "      <td>2472.75</td>\n",
       "      <td>B.1.1.7 S-2p</td>\n",
       "      <td>IgA</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>Control VLP</td>\n",
       "      <td>H46L</td>\n",
       "      <td>B.1.1.7 S-2p_IgA</td>\n",
       "      <td>3097.75</td>\n",
       "      <td>B.1.1.7 S-2p</td>\n",
       "      <td>IgA</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>Control VLP</td>\n",
       "      <td>H47</td>\n",
       "      <td>B.1.1.7 S-2p_IgA</td>\n",
       "      <td>2827.00</td>\n",
       "      <td>B.1.1.7 S-2p</td>\n",
       "      <td>IgA</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"

  },
  {
   "cell_type": "code",
   "execution_count": 41,
   "id": "02241a80-73b2-42d1-b2ca-b1f4396e75f0",
   "metadata": {},
   "outputs": [],
   "source": [
    "@pn.depends(ag_select.param.value, \n",
    "            ig_fcr_choose.param.value)\n",
    "def ag_strip_plot(antigen=df_plot.Ag.values[0], ig_or_fcr=\"Ig Titer\"):\n",
    "        \n",
    "    if ig_or_fcr == \"Ig Titer\":\n",
    "        df_small = df_plot.loc[(df_plot.Ag == antigen) & (df_plot.Ig_FcR.str.contains(\"Ig\")), :]\n",
    "    else:\n",
    "        df_small = df_plot.loc[(df_plot.Ag == antigen) & (df_plot.Ig_FcR.str.contains(\"Fc\")), :]\n",
    "        \n",
    "    strip = hv.Scatter(\n",
    "        data=df_small,\n",
    "        kdims=['Ig_FcR'],\n",
    "        vdims=[\"value\", \"Treatment\"],\n",
    "    ).opts(\n",
    "        color='Treatment',\n",
    "        jitter=0.3,\n",
    "        size=7,\n",
    "        xlabel=\"\",\n",
    "        ylabel='Median Fluorescence',\n",
    "        logy=True,\n",
    "        width=600,\n",
    "        height=400,\n",
    "        legend_position=\"top\"\n",
    "    )\n",
    "    \n",
    "    tooltips = [('Sample', '@Sample')]\n",
    "    hover = bokeh.models.HoverTool(tooltips=tooltips)\n",
    "    strip.opts(tools=[hover])\n",
    "    \n",
    "    return strip"
   ]

In [ ]:
   "@pn.depends(ab_fcr_select.param.value)\n",
    "def ig_fcr_strip_plot(ig_or_fcr=df_plot.Ig_FcR.values[0]):\n",
    "        \n",
    "    df_small = df_plot.loc[(df_plot.Ig_FcR == ig_or_fcr), :]\n",
    "        \n",
    "    strip = hv.Scatter(\n",
    "        data=df_small,\n",
    "        kdims=['Ag'],\n",
    "        vdims=[\"value\", \"Treatment\"],\n",
    "    ).opts(\n",
    "        color='Treatment',\n",
    "        jitter=0.3,\n",
    "        size=7,\n",
    "        xlabel=\"\",\n",
    "        ylabel='Median Fluorescence',\n",
    "        logy=True,\n",
    "        width=800, \n",
    "        height=400,\n",
    "        legend_position=\"top\"\n",
    "    )\n",
    "    \n",
    "    tooltips = [('Sample', '@Sample')]\n",
    "    hover = bokeh.models.HoverTool(tooltips=tooltips)\n",
    "    strip.opts(tools=[hover])\n",
    "    \n",
    "    return strip"
